<a href="https://colab.research.google.com/github/hamza123232/Machine-Learning/blob/main/t5_2_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.2 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00


In [7]:
import pandas as pd



data_values = []
data_qas = []
data_qas_answer = []
data_set = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data.json')
for n in data_set['context']:
    data_values.append({'context': n})
for i in data_set['qas']:
    for n in i:
        for a in n['answers']:
            data_qas.append(a)
        data_qas_answer.append(n)
data_values_df = pd.DataFrame(data_values)
data_qas_answer_df = pd.DataFrame(data_qas_answer)
data_qas_answer_df.drop('answers', axis=1, inplace=True)
merged_df = data_values_df.join(data_qas_answer_df)
prompts = merged_df.join(pd.DataFrame(data_qas))
prompts = prompts.drop(['id', 'is_impossible', 'answer_start'], axis=1)
prompts['input_text'] = prompts.apply(lambda row: f"{row['context']} {row['question']} {row['text']}", axis=1)

prompts_file = prompts.to_csv("prompts_dataset.txt", header=False, index=False)


In [ ]:
list_a = [1, 2, 3, 4, 5]
list_a.sort(lambda x: x-1)

,context,qas
0,In this module MonetX is fetching data of conf...,"[{'id': '00001', 'is_impossible': True, 'quest..."
1,In the monitoring module MonetX is monitoring ...,"[{'id': '00002', 'is_impossible': True, 'quest..."
2,In IPAM module MonetX is scanning through NMAP...,"[{'id': '00004', 'is_impossible': True, 'quest..."
3,In IPAM dashboard there is widget called Ip av...,"[{'id': '00005', 'is_impossible': True, 'quest..."
4,In IPAM dashboard there is widget called subne...,"[{'id': '00006', 'is_impossible': True, 'quest..."
...,...,...
70,In IPAM montX have ability to add DNS server a...,"[{'id': '00104', 'is_impossible': True, 'quest..."
71,Data Center Capacity Management\nIn DCM module...,"[{'id': '00105', 'is_impossible': True, 'quest..."
72,Network monitoring is the use of a system that...,"[{'id': '00106', 'is_impossible': True, 'quest..."
73,in admin page you can see the failed devices a...,"[{'id': '00107', 'is_impossible': True, 'quest..."


In [8]:
prompts = pd.read_csv("prompts_dataset.txt",names=["context","question", "text",'input_text'])


In [9]:
prompts

,context,question,text,input_text
0,In this module MonetX is fetching data of conf...,What is use of IPAM in MonetX?,In this module MonetX is fetching data of conf...,In this module MonetX is fetching data of conf...
1,In the monitoring module MonetX is monitoring ...,How are top 10 subnet by IP address are displa...,In the monitoring module MonetX is monitoring ...,In the monitoring module MonetX is monitoring ...
2,In IPAM module MonetX is scanning through NMAP...,What is TCP open port graph displaying?,In IPAM module MonetX is Scanning through NMAP...,In IPAM module MonetX is scanning through NMAP...
3,In IPAM dashboard there is widget called Ip av...,What is Ip availability summary showing?,In IPAM dashboard there is widget called Ip av...,In IPAM dashboard there is widget called Ip av...
4,In IPAM dashboard there is widget called subne...,What is subnet summary showing?,In IPAM dashboard there is widget called subne...,In IPAM dashboard there is widget called subne...
...,...,...,...,...
70,In IPAM montX have ability to add DNS server a...,DNS Services mean?,In IPAM montX have ability to add DNS server a...,In IPAM montX have ability to add DNS server a...
71,Data Center Capacity Management\nIn DCM module...,What is DCM?,Data Center Capacity Management\nIn DCM module...,Data Center Capacity Management\nIn DCM module...
72,Network monitoring is the use of a system that...,What is monitoring?,Network monitoring is the use of a system that...,Network monitoring is the use of a system that...
73,in admin page you can see the failed devices a...,What do failed devices display?,in admin page you can see the failed devices a...,in admin page you can see the failed devices a...


In [10]:
import re
def clean_text(text):
    text = re.sub(r',', '', text)
    text = re.sub(r'\'', '', text)
    text = re.sub(r'\"', '', text)
    text = re.sub(r'\(', '', text)
    text = re.sub(r'\)', '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'“', '', text)
    text = re.sub(r'”', '', text)
    text = re.sub(r'’', '', text)
    text = re.sub(r'\.', '', text)
    text = re.sub(r';', '', text)
    text = re.sub(r':', '', text)
    text = re.sub(r'\-', '', text)
    return text

In [11]:
prompts['context'] = prompts['context'].apply(clean_text)

prompts['question'] = prompts['question'].apply(clean_text)
prompts['text'] = prompts['text'].apply(clean_text)
prompts['input_text'] = prompts['input_text'].apply(clean_text)

In [12]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Adafactor

batch_size = 8
num_of_epochs = 10

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base', return_dict=True)

model.to(device)

Running on the GPU


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [13]:
optimizer = Adafactor(model.parameters(), 
                      lr=1e-3, 
                      eps=(1e-30, 1e-3), 
                      clip_threshold=1.0,
                      decay_rate=-0.8, 
                      beta1=None, 
                      weight_decay=0.0, 
                      relative_step=False,
                      scale_parameter=False, 
                      warmup_init=False)

model.train()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [14]:
def progress(loss, value, max=100, bar_width=50):
    progress_value = int(value * bar_width / max)
    bar = '[' + '#' * progress_value + ' ' * (bar_width - progress_value) + ']'
    return f"Batch loss: {loss}\n{bar} {value}/{max}"


In [15]:
loss_per_10_steps = []
num_of_epochs = 20
for epoch in range(1, num_of_epochs + 1):
    print(f"Running epoch: {epoch}")
    running_loss = 0
    num_of_batches = len(prompts) // batch_size
    out = display(progress(1, num_of_batches+1), display_id=True)
    for i in range(num_of_batches):
        input_batch = []
        label_batch = []
        new_df = prompts[i * batch_size : i * batch_size + batch_size]

        for _, row in new_df.iterrows():
            input_text = "WebNLG: " + row["question"] + "</s>"
            target_text = row["text"] + "</s>"
            input_batch.append(input_text)
            label_batch.append(target_text)

        input_batch = tokenizer.batch_encode_plus(input_batch, padding=True, max_length=400, return_tensors="pt")["input_ids"]
        label_batch = tokenizer.batch_encode_plus(label_batch, padding=True, max_length=400, return_tensors="pt")["input_ids"]
        input_batch = input_batch.to(device)
        label_batch = label_batch.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_batch, labels=label_batch)
        loss = outputs.loss
        loss_num = loss.item()
        logits = outputs.logits
        running_loss += loss_num

        if i % 10 == 0:
            loss_per_10_steps.append(loss_num)

        out.update(progress(loss_num, i, num_of_batches + 1))

        loss.backward()

        optimizer.step()

    running_loss = running_loss / int(num_of_batches)
    print(f"Epoch: {epoch}, Running loss: {running_loss}")

Running epoch: 1


'Batch loss: 2.591883659362793\n[########################################          ] 8/10'

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2382: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Epoch: 1, Running loss: 4.252227624257405
Running epoch: 2


'Batch loss: 1.8471605777740479\n[########################################          ] 8/10'

...

Epoch: 18, Running loss: 0.03533844277262688
Running epoch: 19


'Batch loss: 0.04312380030751228\n[########################################          ] 8/10'

Epoch: 19, Running loss: 0.03755969005740351
Running epoch: 20


'Batch loss: 0.04590422287583351\n[########################################          ] 8/10'

Epoch: 20, Running loss: 0.04659175986631049


In [16]:
model.save_pretrained('/content/t5_text_model')
tokenizer.save_pretrained('/content/t5_text_tokenizer')

('/content/t5_text_tokenizer/tokenizer_config.json',
 '/content/t5_text_tokenizer/special_tokens_map.json',
 '/content/t5_text_tokenizer/spiece.model',
 '/content/t5_text_tokenizer/added_tokens.json')

In [17]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_directory = '/content/t5_text_model'
tokenizer_directory = '/content/t5_text_tokenizer'

tokenizer = T5Tokenizer.from_pretrained(tokenizer_directory)
model = T5ForConditionalGeneration.from_pretrained(model_directory, return_dict=True)


In [18]:
!pip install shutil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil


In [ ]:
import shutil
shutil.make_archive('t55_text_model', 'zip', model_directory)

'/content/t55_text_model.zip'

In [ ]:
shutil.make_archive('/content/model_and_tokenizer.h5', 'zip', tokenizer_directory)

'/content/model_and_tokenizer.h5.zip'

In [19]:
def generate(text,model,tokenizer):
   model.eval()
   input_ids = tokenizer.encode("WebNLG:{} </s>".format(text), 
                               return_tensors="pt")  #"WebNLG:{} </s>".format()
   outputs = model.generate(input_ids)
   return tokenizer.decode(outputs[0])

In [20]:
def generate_text(prompt, model, tokenizer,max_length,temperature):
    model.eval()
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(input_ids,do_sample=True, max_length=max_length,temperature=temperature)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_text = generated_text[:max_length]
    return generated_text

generated_text = generate_text("what is montex?", model, tokenizer, max_length=900,temperature=1.0)
print(generated_text)

In this module MonetX is fetching data of configured IPAM and fetching information of configured IPAM it is also s the interface description vlan name subnet Subnets mask and the status of the physical interface up or down The benefit of this module is to track any ip from the dashboard & does not need to login to device in monetX it have capability to add a single device and fetch the information of all ip from the dashboard & does not need to login to device in monetX it is also the interface description if these is a double ip from the dashboard it will show the status of the physical interface information of each ip from the dashboard if these is a double ip from the dashboard if these is a double ip from the dashboard it will show the count of all if the there is a single device it will show the count of the ip from the dashboard if there is a single if there is a separate a separat


In [21]:
import os
import h5py
import base64
from transformers import T5ForConditionalGeneration, T5TokenizerFast

model_directory = '/content/t5_text_model'
tokenizer_directory = '/content/t5_text_tokenizer'
output_h5_file = '/content/model_and_tokenizer.h5'

tokenizer = T5TokenizerFast.from_pretrained(tokenizer_directory)
model = T5ForConditionalGeneration.from_pretrained(model_directory, return_dict=True)

# Create an H5 file
with h5py.File(output_h5_file, 'w') as hf:
    # Save the model weights
    model_state_dict = model.state_dict()
    for key in model_state_dict:
        hf.create_dataset(f'model/{key}', data=model_state_dict[key].numpy())

    # Save the tokenizer options
    tokenizer_options = tokenizer.init_kwargs['additional_special_tokens']
    hf.attrs['tokenizer/options'] = str(tokenizer_options)

    # Save the tokenizer special tokens
    special_tokens = tokenizer.special_tokens_map
    for token_name, token_value in special_tokens.items():
        hf.attrs[f'tokenizer/special_tokens/{token_name}'] = str(token_value)

    # Save the tokenizer files
    tokenizer_files = os.listdir(tokenizer_directory)
    for file_name in tokenizer_files:
        file_path = f"{tokenizer_directory}/{file_name}"
        with open(file_path, 'rb') as file:
            file_data = file.read()
            encoded_data = base64.b64encode(file_data).decode('utf-8')
            hf.create_dataset(f'tokenizer/{file_name}', data=encoded_data)

print(f"Model and tokenizer saved to {output_h5_file}")


Model and tokenizer saved to /content/model_and_tokenizer.h5


In [23]:
import shutil
shutil.copy('/content/model_and_tokenizer.h5', '/content/drive/MyDrive/Colab Notebooks')


'/content/drive/MyDrive/Colab Notebooks/model_and_tokenizer.h5'